# Spherical shell and tangent cylinder analysis

In [1]:
import numpy as np
import glob
import os
import subprocess
import time
from datetime import datetime
import netCDF4 as nc
from scipy.integrate import simps
from numpy.polynomial.legendre import leggauss
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter
from scipy.special import roots_legendre
import gc

In [2]:
def basic_settings():
    """
    Returns:
        tuple: A tuple containing the input path and output path as strings.
    """
    
    # Path to the folder containing input files
    input_path = "C:/path/to/input/" 
    output_path = "C:/path/to/output/"
    
    # Path to the folder containing input files for testing on PC
    #input_path = "C:/Users/scjkk/Desktop/Test_data/Ra30_files/" 
    #output_path = "C:/Users/scjkk/Desktop/Test_data/Ra30_files/output/" 
    
    # real versions
    #input_path = "/nfs/a88/earcd/INHOMOGENEOUS_ICB/E=1e-6/Pm=0/Pr=1.0/HOMOGENEOUS/Ra=2000/STATEFILES/" 
    #output_path = "/nfs/see-fs-01_users/scjkk/output_folder/" 
    
    return input_path, output_path

In [3]:
def create_links(input_path):
    """
    Copies files matching a specific pattern from a remote location to the current directory
    and returns a list of source file paths.

    Args:
        input_path (str): The path to the directory containing the files to copy.

    Returns:
        list: A list of source file paths copied.
    """
    
    # Initialize lists to store paths and source file names
    input_list_paths = []
    input_list_source = []
    
    # Find all files in the input path matching the pattern *state00501*T00=0*
    remote_files = glob.glob(input_path + "*state00501*T00=0*")  # Essential part of input filename. 
    
    # Copy each file from the remote location to the current directory
    for file in remote_files:
        input_list_paths.append(subprocess.run(['scp', '-qC', file, '.']))  # Execute SCP command to copy file
        input_list_source.append(file)  # Add source file path to the list
    
    return input_list_source

In [4]:
def create_input_list(input_path):
    """
    Generates a sorted list of input file names from a specified directory.

    Args:
        input_path (str): The path to the directory containing the files.

    Returns:
        list: A sorted list of base names of files matching the pattern.
    """
    
    # Initialize lists to store file paths and file names
    input_list_paths = []
    input_list = []
    
    # Find all files in the input path matching the pattern *state00501*T00=0*
    for timestep in glob.glob(input_path + '*state00501*T00=0*'):  # Insert essential part of input filename
        input_list_paths.append(timestep)  # Add file path to the list of paths
    
    # Extract base names from the file paths
    for i in range(len(input_list_paths)):
        input_list.append(os.path.basename(input_list_paths[i]))  # Extract base name and add to the list
    
    # Sort the list of base names
    input_list.sort()
    
    return input_list


In [5]:
def get_data(coreData):
    """
    Extracts and processes data from a netCDF4 dataset.

    This function retrieves various variables from the provided netCDF4 dataset,
    processes them (including reversing latitude arrays for polar coordinates), 
    and returns the processed data.

    Args:
        coreData (netCDF4.Dataset): The netCDF4 dataset containing the required variables.

    Returns:
        tuple: Contains the following elements:
            el (ndarray): Reversed latitude array for polar coordinates.
            r (ndarray): Radial coordinate array.
            az (ndarray): Azimuthal coordinate array.
            temp (ndarray): Temperature array with reversed latitude.
            u_r (ndarray): Radial velocity array with reversed latitude.
            u_el (ndarray): Latitudinal velocity array with reversed latitude.
            u_az (ndarray): Azimuthal velocity array with reversed latitude.
            d_T_del (ndarray): Temperature gradient in the latitudinal direction with reversed latitude.
            d_T_dr (ndarray): Temperature gradient in the radial direction with reversed latitude.
    """
    # Extract data from coreData variables
    r = np.ma.getdata(coreData.variables['r'][:])
    az = np.ma.getdata(coreData.variables['ph'][:])
    el1 = np.ma.getdata(coreData.variables['th'][:])
    temp1 = np.ma.getdata(coreData.variables['C'][:])
    u_r1 = np.ma.getdata(coreData.variables['ur'][:])
    u_el1 = np.ma.getdata(coreData.variables['uth'][:])
    u_az1 = np.ma.getdata(coreData.variables['uph'][:])
    d_T_del1 = np.ma.getdata(coreData.variables['Ct'][:])
    d_T_dr1 = np.ma.getdata(coreData.variables['Cr'][:])

    # Reverse latitude arrays to convert to polar coordinates
    el = el1[::-1]
    u_r = u_r1[:, :, ::-1]
    u_az = u_az1[:, :, ::-1]
    u_el = u_el1[:, :, ::-1]
    temp = temp1[:, :, ::-1]
    d_T_del = d_T_del1[:, :, ::-1]
    d_T_dr = d_T_dr1[:, :, ::-1]

    # Return the processed variables
    return el, r, az, temp, u_r, u_el, u_az, d_T_del, d_T_dr


### create grid for spherical coordinates



In [6]:
def create_grid(r, az, el):
    """
    Creates a 3D grid from given radial, azimuthal, and latitudinal coordinates.

    This function generates a flattened 3D grid using the provided radial, azimuthal,
    and latitudinal coordinates. It also creates masks for the core-mantle boundary (CMB)
    and the inner-core boundary (ICB).

    Args:
        r (ndarray): Array of radial coordinates.
        az (ndarray): Array of azimuthal coordinates.
        el (ndarray): Array of latitudinal coordinates.

    Returns:
        tuple: Contains the following elements:
            r1 (ndarray): Flattened array of radial coordinates.
            az1 (ndarray): Flattened array of azimuthal coordinates.
            el1 (ndarray): Flattened array of latitudinal coordinates.
            cmb_mask (ndarray): Mask array for core-mantle boundary (CMB).
            icb_mask (ndarray): Mask array for inner-core boundary (ICB).
    """
    # Calculate the total number of grid points
    total_points = len(r) * len(az) * len(el)

    # Initialize flattened arrays for coordinates
    r1 = np.zeros(total_points)
    az1 = np.zeros(total_points)
    el1 = np.zeros(total_points)

    # Fill the arrays with corresponding coordinates
    index = 0
    for i in r:
        for j in az:
            for k in el:
                r1[index] = i
                az1[index] = j
                el1[index] = k
                index += 1

    # Create masks for core-mantle boundary (CMB) and inner-core boundary (ICB)
    cmb_mask = r1 > r.max() - 0.00001
    icb_mask = r1 < r.min() + 0.00001

    return r1, az1, el1, cmb_mask, icb_mask


### Kinetic energy, Energy of viscosity, lengthscale, Reynolds number: spherical shell

$KE = \frac{1}{2} \int_V |u|^2\ dV$

$EOV = \frac{1}{2} \int_V |\omega|^2\ dV$

Lengthscale $L = \sqrt{\frac{KE}{EOV}}$

$Re =  \sqrt{\frac{2 KE}{V}}$

In [7]:
def KE1(u_r, u_az, u_el, r, el):
    """
    Calculates the kinetic energy integrand for a given velocity field.

    This function computes the kinetic energy integrand based on the radial, azimuthal,
    and latitudinal components of the velocity field, along with the radial and 
    latitudinal coordinates.

    Args:
        u_r (ndarray): Radial component of the velocity field.
        u_az (ndarray): Azimuthal component of the velocity field.
        u_el (ndarray): Latitudinal component of the velocity field.
        r (ndarray): Radial coordinates.
        el (ndarray): Latitudinal coordinates.

    Returns:
        ndarray: The kinetic energy integrand.
    """
    # Calculate the squared sum of the velocity components
    velocity_squared_sum = u_r**2 + u_az**2 + u_el**2

    # Calculate the kinetic energy integrand using the formula
    KE_integrand = r**2 * np.sin(el) * velocity_squared_sum

    return KE_integrand


### Vorticity

$\omega_r = \frac{1}{r \sin{\theta}}  \left(  \frac{\partial }{\partial \theta} (u_\phi \sin{\theta})  - \frac{\partial u_\theta}{\partial \phi} \right) $

$\omega_{\theta} = \frac{1}{r}  \left( \frac{1}{\sin{\theta}}  \frac{\partial u_r}{\partial \phi} - \frac{\partial }{\partial r} (r u_\phi) \right) $

$\omega_{\phi} = \frac{1}{r} \left( \frac{\partial }{\partial r} (r u_\theta) - \frac{\partial u_r}{\partial \theta}   \right) $


In [8]:
def calculate_vorticity(u_r, u_el, u_az, r2, el2, az2, r, el, az):
    """
    Computes vorticity components from a velocity field in spherical coordinates.

    Args:
        u_r (ndarray): Radial velocity component.
        u_el (ndarray): Latitudinal velocity component.
        u_az (ndarray): Azimuthal velocity component.
        r2 (ndarray): Radial coordinates for vorticity calculation.
        el2 (ndarray): Latitudinal coordinates for vorticity calculation.
        az2 (ndarray): Azimuthal coordinates for vorticity calculation.
        r (ndarray): Radial coordinates.
        el (ndarray): Latitudinal coordinates.
        az (ndarray): Azimuthal coordinates.

    Returns:
        tuple: Vorticity components (omega_r, omega_el, omega_az).
    """
    # Ensure the input arrays have the correct shapes
    assert u_r.shape == u_el.shape == u_az.shape == r2.shape == el2.shape == az2.shape,\
        "All input arrays must have the same shape"
    
    # Precompute sines and cosines
    sin_el = np.sin(el2)
    cos_el = np.cos(el2)
    
    # Initialize vorticity components with the same shape as input velocities
    omega_r = np.zeros_like(u_r)
    omega_el = np.zeros_like(u_el)
    omega_az = np.zeros_like(u_az)
    
    # Calculate derivatives
    d_u_r_del = np.gradient(u_r, el, axis=2)
    d_u_r_daz = np.gradient(u_r, az, axis=1)
    d_u_el_dr = np.gradient(u_el, r, axis=0)
    d_u_el_daz = np.gradient(u_el, az, axis=1)
    d_u_az_dr = np.gradient(u_az, r, axis=0)
    d_u_az_del = np.gradient(u_az, el, axis=2)
    
    # Calculate vorticity components
    omega_r = (1 / (r2 * sin_el)) * (d_u_az_del * sin_el - d_u_el_daz)
    omega_el = (1 / r2) * (1 / sin_el * d_u_r_daz - np.gradient(r2 * u_az, r, axis=0))
    omega_az = (1 / r2) * (d_u_el_dr - d_u_r_del)
    
    return omega_r, omega_el, omega_az


In [9]:
def KE_EOV_Re(input_list, r1, az1, el1, u_r, u_el, u_az, r, az, el):
    """
    Calculates the kinetic energy (KE), energy of vorticity (EOV), lengthscale (L),
    and Reynolds number (Re) for a given velocity field.

    Args:
        input_list (list): List of input data files.
        r1 (ndarray): Flattened radial coordinates.
        az1 (ndarray): Flattened azimuthal coordinates.
        el1 (ndarray): Flattened latitudinal coordinates.
        u_r (ndarray): Radial component of the velocity field.
        u_el (ndarray): Latitudinal component of the velocity field.
        u_az (ndarray): Azimuthal component of the velocity field.
        r (ndarray): Radial coordinates.
        az (ndarray): Azimuthal coordinates.
        el (ndarray): Latitudinal coordinates.

    Returns:
        tuple: (KE, EOV, L, Re)
            KE (float): Kinetic energy.
            EOV (float): Energy of vorticity.
            L (float): Lengthscale.
            Re (float): Reynolds number.
    """
    # Reshape the gridded coordinate arrays to match the velocity field shape
    r2 = r1.reshape(len(r), len(az), len(el))
    el2 = el1.reshape(len(r), len(az), len(el))
    az2 = az1.reshape(len(r), len(az), len(el))

    # Calculate the kinetic energy integrand
    integrand1 = KE1(u_r, u_az, u_el, r2, el2)

    # Integrate to find KE
    integralKE = simps([simps([simps(value, el) for value in integrand1], az)], r)
    KE = 0.5 * integralKE

    # Calculate vorticity components
    omega_r, omega_el, omega_az = calculate_vorticity(u_r, u_el, u_az, r2, el2, az2, r, el, az)

    # Calculate the energy of vorticity integrand
    integrand2 = KE1(omega_r, omega_az, omega_el, r2, el2)

    # Integrate to find EOV
    integralEOV = simps([simps([simps(value, el) for value in integrand2], az)], r)
    EOV = 0.5 * integralEOV

    # Calculate lengthscale
    L = np.sqrt(KE / EOV)

    # Calculate Reynolds number
    shell_volume = (4 / 3 * np.pi * r.max()**3) - (4 / 3 * np.pi * r.min()**3)
    Re = np.sqrt((2 * KE) / shell_volume)

    return KE, EOV, L, Re


### Find Nusselt number: spherical shell

$ Nu = \frac{1.209}{\Delta T}$

In [10]:
def spherical_to_cartesian(radius, el, az):
    """
    Converts spherical coordinates to Cartesian coordinates.

    Args:
        radius (ndarray): Radial coordinates.
        el (ndarray): Elevation coordinates (in radians).
        az (ndarray): Azimuth coordinates (in radians).

    Returns:
        tuple: (x, y, z)
            x (ndarray): Cartesian x-coordinates.
            y (ndarray): Cartesian y-coordinates.
            z (ndarray): Cartesian z-coordinates.
    """
    # Calculate the Cartesian coordinates
    x = radius * np.sin(el) * np.cos(az)
    y = radius * np.sin(el) * np.sin(az)
    z = radius * np.cos(el)
    
    return x, y, z


In [11]:
def nodes_weights_spherical(order):
    """
    Calculates nodes and weights for spherical Gaussian quadrature.

    Args:
        order (int): Order of the Gaussian quadrature.

    Returns:
        tuple: (nodes_cart, weights_norm, nodes1, weights1)
            nodes_cart (ndarray): Cartesian coordinates of the quadrature nodes.
            weights_norm (ndarray): Normalized weights for the quadrature nodes.
            nodes1 (ndarray): Nodes for Gaussian quadrature in the interval [-1, 1].
            weights1 (ndarray): Weights for Gaussian quadrature in the interval [-1, 1].
    """
    # Nodes and weights for Gaussian quadrature in the interval [-1, 1]
    nodes1, weights1 = leggauss(order)

    # Azimuthal nodes in the interval [0, pi]
    phi = np.pi * (np.arange(1, 2 * order + 1) - 0.5) / order
    
    # Initialize Cartesian coordinates array
    nodes_cart = np.zeros((2 * order * order, 3))
    
    # Convert spherical coordinates to Cartesian coordinates
    count = 0
    for i in range(order):
        for j in range(2 * order):
            nodesi = nodes1[i]
            phij = phi[j]
            sqrt_term = np.sqrt(1 - nodesi ** 2)
            nodes_cart[count] = [sqrt_term * np.cos(phij), sqrt_term * np.sin(phij), nodesi]
            count += 1

    # Initialize weights array
    weights = np.zeros(2 * order * order)
    
    # Assign weights, ensuring angular coverage
    for i in range(order):
        for j in range(2 * order):
            weights[i * 2 * order + j] = 2 * np.pi / order * weights1[i]

    # Normalize the weights to sum to 4*pi
    weights_norm = weights * 4 * np.pi / weights.sum()
    
    return nodes_cart, weights_norm, nodes1, weights1


In [12]:
def DeltaT(temp, r, az, el, r1, az1, el1, cmb_mask, icb_mask, order):
    """
    Calculates the temperature difference (Delta_T) between the core-mantle boundary (CMB) 
    and the inner-core boundary (ICB) using Gaussian quadrature.

    Args:
        temp (ndarray): Temperature field.
        r (ndarray): Radial coordinates.
        az (ndarray): Azimuthal coordinates.
        el (ndarray): Latitudinal coordinates.
        r1 (ndarray): Flattened radial coordinates.
        az1 (ndarray): Flattened azimuthal coordinates.
        el1 (ndarray): Flattened latitudinal coordinates.
        cmb_mask (ndarray): Mask indicating core-mantle boundary coordinates.
        icb_mask (ndarray): Mask indicating inner-core boundary coordinates.
        order (int): Order of the Gaussian quadrature.

    Returns:
        float: The temperature difference (Delta_T) between the CMB and ICB.
    """
    # Extract elevation and azimuthal coordinates for CMB and ICB
    el_cmb, el_icb = el1[cmb_mask], el1[icb_mask]
    az_cmb, az_icb = az1[cmb_mask], az1[icb_mask]

    # Flatten temperature array and extract values for CMB and ICB
    temp_flat = temp.flatten()
    temp_cmb = temp_flat[cmb_mask].reshape(len(az), len(el))
    temp_icb = temp_flat[icb_mask].reshape(len(az), len(el))

    # Gaussian quadrature nodes and weights
    nodes_cart, weights_norm, _, _ = nodes_weights_spherical(order)

    # Convert spherical to Cartesian coordinates for CMB and ICB
    coords_cart_cmb = np.array([spherical_to_cartesian(r.max(), t, p) for t, p in zip(el_cmb, az_cmb)])
    coords_cart_icb = np.array([spherical_to_cartesian(r.min(), t, p) for t, p in zip(el_icb, az_icb)])

    # Interpolate temperature data onto the nodes in Cartesian coordinates
    interp_scalar_data_cmb = griddata(coords_cart_cmb, temp_flat[cmb_mask], nodes_cart, method='nearest', fill_value=0.0)
    interp_scalar_data_icb = griddata(coords_cart_icb, temp_flat[icb_mask], nodes_cart, method='nearest', fill_value=0.0)

    # Integrate interpolated data using Gaussian quadrature
    temp_av_cmb = np.sum(interp_scalar_data_cmb * weights_norm) * r.max()**2
    temp_av_icb = np.sum(interp_scalar_data_icb * weights_norm) * r.min()**2

    # Average temperature by surface area
    av_cmb = temp_av_cmb / (r.max()**2 * 4 * np.pi)
    av_icb = temp_av_icb / (r.min()**2 * 4 * np.pi)

    # Calculate temperature difference (Delta_T)
    Delta_T = abs(av_cmb - av_icb)
    
    return Delta_T


In [13]:
def find_Nu_global( r, Delta_T):
    """
    Calculates the global Nusselt number (Nu_shell) for a given temperature difference.

    Args:
        r (ndarray): Radial coordinates.
        Delta_T (float): Temperature difference between the core-mantle boundary (CMB) 
                         and the inner-core boundary (ICB).

    Returns:
        float: The global Nusselt number (Nu_shell).
    """
    # Precompute the constant deltaTC using the maximum and minimum radial coordinates
    deltaTC = 1 / (r.max() * r.min())
    
    # Calculate the global Nusselt number
    Nu_shell = deltaTC / Delta_T
    
    return Nu_shell


### Create cylindrical grid

In [14]:
#fixed 2

def output_grid(r, num):
    """
    Generates a cylindrical grid with specified radial, angular, and vertical coordinates.

    Args:
        r (ndarray): Radial coordinates to determine cylinder dimensions.
        num (int): Number of grid points for each coordinate.

    Returns:
        tuple: Radial (s), angular (phi), and vertical (z) coordinates for the grid.
    """
    # Generate radial coordinates from the radius divided by num to the TC sidewall
    s = np.linspace(r.min() / num, r.min(), num)
    
    # Wide cylinder
    #s = np.linspace(r.min()/num, 0.55, num)
    
    # Narrow cylinder
    #s = np.linspace(r.min()/num, 0.4, num)
    
    # Calculate the maximum radius and corresponding thresholds
    radius = s.max()
    theta_threshold_TC = np.arcsin(radius / r.max())  # Theta at TC boundary on CMB
    phi_threshold = np.pi / 2 - theta_threshold_TC
    height_at_boundary = r.max() * np.sin(phi_threshold)
    
    # Generate z coordinates from 0 to the height at the sidewall
    z = np.linspace(0.0, height_at_boundary, num)
    
    # Generate phi coordinates for a full circle
    phi = np.linspace(0, 2 * np.pi, num, endpoint=False)
    
    
    return s, phi, z


In [15]:
def cyl2sph(s, phi, z):
    """
    Converts cylindrical coordinates to spherical coordinates.

    Args:
        s (ndarray): Radial distance in cylindrical coordinates.
        phi (ndarray): Azimuthal angle in radians in cylindrical coordinates.
        z (ndarray): Vertical coordinate in cylindrical coordinates.

    Returns:
        tuple: Spherical coordinates (r, az, el).
    """
    # Calculate radial distance in spherical coordinates
    r = np.sqrt(s**2 + z**2)
    
    # Azimuthal angle remains the same
    az = phi
    
    # Calculate elevation angle in spherical coordinates
    el = np.arccos(np.clip(z / r, -1, 1))
    
    return r, az, el


In [16]:
def convert_vec_sph2cyl(az, el, u_r, u_az, u_el):
    """
    Converts velocity components from spherical to cylindrical coordinates.

    Args:
        az (ndarray): Azimuthal angle in radians.
        el (ndarray): Elevation angle in radians.
        u_r (ndarray): Radial velocity component.
        u_az (ndarray): Azimuthal velocity component.
        u_el (ndarray): Elevation velocity component.

    Returns:
        tuple: Velocity components in cylindrical coordinates (u_s, u_phi, u_z).
    """
    # Precompute trigonometric functions
    sin_el = np.sin(el)
    cos_el = np.cos(el)
    sin_az = np.sin(az)
    cos_az = np.cos(az)
    
    # Convert spherical components to Cartesian components
    ux = u_r * sin_el * cos_az
    uy = u_r * sin_el * sin_az
    uz = u_r * cos_el - u_el * sin_el  

    # Convert Cartesian components to cylindrical components
    u_s = ux * cos_az + uy * sin_az  # Radial component in cylindrical coordinates
    u_phi = u_az  # Azimuthal component remains unchanged
    u_z = uz  # Axial component

    return u_s, u_phi, u_z


In [17]:
def interpolation_TC_grid(num, r, az, el, temp, u_r, u_el, u_az, s, phi, z, r1, az1, el1, d_T_dr):
    """
    Interpolates spherical grid data onto a cylindrical grid and converts velocity components.

    Args:
        num (int): Number of divisions for the cylindrical grid.
        r, az, el (ndarray): Spherical coordinates.
        temp, u_r, u_el, u_az (ndarray): Temperature and velocity components in spherical coordinates.
        s, phi, z (ndarray): Cylindrical coordinates.
        r1, az1, el1 (ndarray): Original spherical grid coordinates.
        d_T_dr (ndarray): Radial temperature gradient in spherical coordinates.

    Returns:
        tuple: Interpolated cylindrical grid data and masks.
            s_tc, phi_tc, z_tc (ndarray): Cylindrical grid coordinates.
            u_s, u_phi, u_z (ndarray): Velocity components in cylindrical coordinates.
            r_tc, az_tc, el_tc (ndarray): Interpolated spherical coordinates.
            ur_tc, uaz_tc, uel_tc (ndarray): Interpolated velocity components.
            temp_tc (ndarray): Interpolated temperature.
            d_T_dr_tc (ndarray): Interpolated radial temperature gradient.
            cyl_top_mask, cyl_base_mask (ndarray): Masks for the top and base of the cylindrical grid.
    """
    # Create a cylindrical grid and flatten
    s_tc, phi_tc, z_tc = np.meshgrid(s, phi, z, indexing='ij')
    s_tc, phi_tc, z_tc = s_tc.flatten(), phi_tc.flatten(), z_tc.flatten()
    
    # Masks for the top and base of the cylindrical grid
    z_upper, z_lower = z.max() + 0.00001, z.max() - 0.00001
    cyl_top_mask = np.logical_and(z_tc < z_upper, z_tc > z_lower)
    z_upper2, z_lower2 = z.min() + 0.00001, z.min() - 0.00001
    cyl_base_mask = np.logical_and(z_tc < z_upper2, z_tc > z_lower2)

    # Convert cylindrical coordinates to spherical coordinates
    r_tc, az_tc, el_tc = cyl2sph(s_tc, phi_tc, z_tc)
    
    # Flatten arrays for interpolation
    ur_flat, uaz_flat, uel_flat = u_r.flatten(), u_az.flatten(), u_el.flatten()
    temp_flat, d_T_dr_flat = temp.flatten(), d_T_dr.flatten()

    # Interpolate values onto the cylindrical grid
    temp_tc = griddata((r1.flatten(), az1.flatten(), el1.flatten()), temp_flat, (r_tc, az_tc, el_tc), method='nearest')
    ur_tc = griddata((r1.flatten(), az1.flatten(), el1.flatten()), ur_flat, (r_tc, az_tc, el_tc), method='nearest')
    uel_tc = griddata((r1.flatten(), az1.flatten(), el1.flatten()), uel_flat, (r_tc, az_tc, el_tc), method='nearest')
    uaz_tc = griddata((r1.flatten(), az1.flatten(), el1.flatten()), uaz_flat, (r_tc, az_tc, el_tc), method='nearest')
    d_T_dr_tc = griddata((r1.flatten(), az1.flatten(), el1.flatten()), d_T_dr_flat, (r_tc, az_tc, el_tc), method='nearest')

    # Convert the interpolated velocity components to cylindrical form
    u_s, u_phi, u_z = convert_vec_sph2cyl(az_tc, el_tc, ur_tc, uaz_tc, uel_tc)

    return s_tc, phi_tc, z_tc, u_s, u_phi, u_z, r_tc, az_tc, el_tc, ur_tc, uaz_tc, uel_tc, temp_tc, d_T_dr_tc,\
                 cyl_top_mask, cyl_base_mask


### Kinetic energy, Energy of viscosity, lengthscale, Reynolds number: tangent cylinder

$KE = \frac{1}{2} \int_V |u|^2\ dV$

$EOV = \frac{1}{2} \int_V |\omega|^2\ dV$

Lengthscale $L = \sqrt{\frac{KE}{EOV}}$

$Re =  \sqrt{\frac{2 KE}{V}}$

In [18]:
def KE2(u_s, u_phi, u_z, s):
    """
    Computes the integrand for kinetic energy in cylindrical coordinates.

    Args:
        u_s (ndarray): Radial velocity component.
        u_phi (ndarray): Azimuthal velocity component.
        u_z (ndarray): Axial velocity component.
        s (ndarray): Radial coordinates.

    Returns:
        ndarray: Kinetic energy integrand.
    """
    # Sum of squared velocity components
    velocity_squared = u_s**2 + u_phi**2 + u_z**2

    # Integrand for kinetic energy
    return s * velocity_squared


### Vorticity (cylindrical basis)

$\omega_s = \frac{\partial u_\phi}{\partial z} - \frac{\partial u_z}{\partial \phi}$

$\omega_\phi = \frac{\partial u_z}{\partial s} - \frac{\partial u_s}{\partial z}$

$\omega_z = \frac{1}{s} \left( \frac{\partial u_s}{\partial \phi} - \frac{\partial (s u_\phi)}{\partial s} \right) $

In [19]:
def calculate_vorticity_cyl(u_s, u_z, u_phi, s_tc, z_tc, phi_tc, z, phi, s):
    """
    Computes vorticity components in cylindrical coordinates.

    Args:
        u_s (ndarray): Radial velocity.
        u_z (ndarray): Axial velocity.
        u_phi (ndarray): Azimuthal velocity.
        s_tc (ndarray): Radial coordinates.
        z_tc (ndarray): Axial coordinates.
        phi_tc (ndarray): Azimuthal coordinates.
        z (ndarray): Axial grid.
        phi (ndarray): Azimuthal grid.
        s (ndarray): Radial grid.

    Returns:
        tuple: (omega_s, omega_z, omega_phi) - Vorticity components.
    """
    # Validate input shapes
    assert u_s.shape == u_z.shape == u_phi.shape == s_tc.shape == z_tc.shape == phi_tc.shape,\
        "All input arrays must have the same shape"

    # Compute gradients
    d_u_s_dz = np.gradient(u_s, z, axis=2)
    d_u_s_dphi = np.gradient(u_s, phi, axis=1)
    d_u_z_ds = np.gradient(u_z, s, axis=0)
    d_u_z_dphi = np.gradient(u_z, phi, axis=1)
    d_u_phi_ds = np.gradient(u_phi, s, axis=0)
    d_u_phi_dz = np.gradient(u_phi, z, axis=2)

    # Calculate vorticity components
    omega_s = d_u_phi_dz - d_u_z_dphi
    omega_z = (1 / s_tc) * (d_u_s_dphi - np.gradient(u_phi * s_tc, s, axis=0))
    omega_phi = d_u_z_ds - d_u_s_dz

    return omega_s, omega_z, omega_phi


In [20]:
def KE_EOV_Re_cyl(s_tc, phi_tc, z_tc, u_s, u_phi, u_z, s, phi, z):
    """
    Computes kinetic energy (KE), energy of vorticity (EOV), length scale (L), and Reynolds number (Re)
    in cylindrical coordinates.

    Args:
        s_tc (ndarray): Flattened radial coordinates.
        phi_tc (ndarray): Flattened azimuthal coordinates.
        z_tc (ndarray): Flattened axial coordinates.
        u_s (ndarray): Radial velocity component.
        u_phi (ndarray): Azimuthal velocity component.
        u_z (ndarray): Axial velocity component.
        s (ndarray): Radial coordinates.
        phi (ndarray): Azimuthal coordinates.
        z (ndarray): Axial coordinates.

    Returns:
        tuple: KE, EOV, length scale (L), Reynolds number (Re).
    """
    # Reshape coordinate and velocity arrays
    s_tc2 = s_tc.reshape(len(s), len(phi), len(z))
    phi_tc2 = phi_tc.reshape(len(s), len(phi), len(z))
    z_tc2 = z_tc.reshape(len(s), len(phi), len(z))
    us = u_s.reshape(len(s), len(phi), len(z))
    uphi = u_phi.reshape(len(s), len(phi), len(z))
    uz = u_z.reshape(len(s), len(phi), len(z))

    # Calculate kinetic energy
    integrand_KE = KE2(us, uphi, uz, s_tc2)
    KE = 0.5 * simps(simps(simps(integrand_KE, z), phi), s)

    # Calculate vorticity components
    omega_s, omega_phi, omega_z = calculate_vorticity_cyl(us, uz, uphi, s_tc2, z_tc2, phi_tc2, z, phi, s)

    # Calculate energy of vorticity
    integrand_EOV = KE2(omega_s, omega_phi, omega_z, s_tc2)
    EOV = 0.5 * simps(simps(simps(integrand_EOV, z), phi), s)

    # Compute length scale and Reynolds number
    L = np.sqrt(KE / EOV)
    cyl_volume = np.pi * s.max()**2 * (z.max() - z.min())
    Re = np.sqrt(2 * KE / cyl_volume)

    return KE, EOV, L, Re


### Find Nusselt number: cylinder

Convective heat flux $C = \frac{ 1}{A_{cmb}} \int \int \int_V \langle u_r T \rangle\ dV$ (from Mound 2017)
  
$ Nu = 1 + \frac{C}{\Delta T}$

(No geometric shortcut for the cylinder)

In [21]:
def cylindrical_to_cartesian(z, s, phi):
    """
    Convert cylindrical coordinates to Cartesian coordinates.

    Args:
        s (ndarray): Radial distances.
        phi (ndarray): Azimuthal angles.
        z (ndarray): Axial positions.

    Returns:
        tuple: Cartesian coordinates (x, y, z).
    """
    x = s * np.cos(phi)
    y = s * np.sin(phi)
    return x, y, z


In [22]:
def nodes_weights_disk(order, s):
    """
    Generate Gaussian quadrature nodes and weights for a flat disk.

    Args:
        order (int): Quadrature order.
        s (ndarray): Radial coordinates.

    Returns:
        tuple: 
            - nodes_cart (ndarray): Cartesian coordinates of the nodes.
            - weights_norm (ndarray): Normalized weights for each node.
    """
    # Legendre polynomial roots and weights
    nodes1, weights1 = roots_legendre(order)
    
    # Disk radius
    radius = s.max()
    
    # Angular nodes (phi)
    phi = np.linspace(0, 2 * np.pi, 2 * order, endpoint=False)
    
    # Initialize node and weight arrays
    num_nodes = 2 * order**2
    nodes_cart = np.zeros((num_nodes, 3))
    weights = np.zeros(num_nodes)
    
    # Calculate nodes and weights
    for i in range(order):
        for j in range(2 * order):
            r = radius * nodes1[i]
            nodes_cart[i * 2 * order + j] = [r * np.cos(phi[j]), r * np.sin(phi[j]), 0]
            weights[i * 2 * order + j] = np.pi / order * weights1[i]

    # Normalize weights to sum to 1 and scale by pi
    weights_norm = weights / np.sum(weights) * np.pi
    
    return nodes_cart, weights_norm


In [23]:
def integrand_uzT(u_z, temp):
    """
    Compute the convective heat flux integrand.

    Args:
        u_z (ndarray): Axial velocity component.
        temp (ndarray): Temperature field.

    Returns:
        ndarray: Convective heat flux integrand (u_z * temp).
    """
    return u_z * temp


In [24]:
def find_uzT_cylinder( u_z, temp_tc, s, phi, z, s_tc, phi_tc, z_tc, cyl_top_mask, order):
    """
    Compute the convective heat flux (uzT) over a cylindrical volume.

    Uses Gauss quadrature for integration over each cylindrical slice, then combines the results.

    Args:
        u_z (ndarray): Axial velocity component.
        temp_tc (ndarray): Temperature field.
        s (ndarray): Radial coordinates in cylindrical system.
        phi (ndarray): Azimuthal coordinates in cylindrical system.
        z (ndarray): Axial coordinates in cylindrical system.
        s_tc (ndarray): Flattened radial coordinates on the cylinder's top surface.
        phi_tc (ndarray): Flattened azimuthal coordinates on the cylinder's top surface.
        z_tc (ndarray): Flattened axial coordinates.
        cyl_top_mask (ndarray): Mask for selecting the cylinder's top surface.
        order (int): Order of the Gauss quadrature.

    Returns:
        float: Total convective heat flux (uzT) for the cylindrical volume.
    """
    # Compute the integrand for convective heat flux
    uzT_data = (integrand_uzT(u_z, temp_tc)).reshape(len(s), len(phi), len(z))
    s_disk = s_tc[cyl_top_mask]
    phi_disk = phi_tc[cyl_top_mask]

    # Gauss quadrature nodes and weights
    nodes_cart, weights_norm = nodes_weights_disk(order, s)

    # Volume of the cylinder
    cylinder_volume = np.pi * s.max()**2 * (z.max() - z.min())

    uzT_disk1 = []

    for z_index in range(len(z)):
        # Convert cylindrical to Cartesian coordinates
        coords_cart = cylindrical_to_cartesian(z[z_index], s_disk, phi_disk)
        uzT_disk = uzT_data[z_index].flatten()

        # Interpolate using nearest neighbor
        interp_scalar_data = griddata(coords_cart, uzT_disk, nodes_cart, method='nearest', fill_value=0.0)

        # Calculate integral for the current z slice
        uzT_1 = np.sum(interp_scalar_data * weights_norm) * s[z_index]**2
        uzT_disk1.append(uzT_1)

    # Integrate over z using Simpson's rule
    return simps(uzT_disk1, z)


In [25]:
def find_DeltaT_cyl(temp_tc, u_z, s, phi, z, s_tc, phi_tc, z_tc, cyl_top_mask, cyl_base_mask, order):
    """
    Compute the temperature difference (DeltaT) between the top and base of a cylinder.

    Calculates average temperatures at the top and base surfaces using Gauss quadrature, then finds their difference.

    Args:
        temp_tc (ndarray): Temperature fields.
        s (ndarray): Radial coordinates.
        phi (ndarray): Azimuthal coordinates.
        z (ndarray): Axial coordinates.
        s_tc (ndarray): Flattened radial coordinates on cylindrical surfaces.
        phi_tc (ndarray): Flattened azimuthal coordinates on cylindrical surfaces.
        z_tc (ndarray): Flattened axial coordinates.
        cyl_top_mask (ndarray): Mask for the top surface.
        cyl_base_mask (ndarray): Mask for the base surface.
        order (int): Gauss quadrature order.

    Returns:
        float: Temperature difference (DeltaT) between the top and base.
    """
    # Flatten and reshape temperature arrays for top and base surfaces
    temp_top = temp_tc[cyl_top_mask].reshape(len(s), len(phi))
    temp_base = temp_tc[cyl_base_mask].reshape(len(s), len(phi))

    # Get Gauss quadrature nodes and weights
    nodes_cart, weights_norm = nodes_weights_disk(order, s)
    
    # Convert cylindrical coordinates to Cartesian for interpolation
    coords_cart_top = cylindrical_to_cartesian(z.max(), s_tc[cyl_top_mask], phi_tc[cyl_top_mask])
    coords_cart_base = cylindrical_to_cartesian(z.min(), s_tc[cyl_base_mask], phi_tc[cyl_base_mask])

    # Interpolate temperatures onto Gauss quadrature nodes
    interp_top = griddata(coords_cart_top, temp_top.flatten(), nodes_cart, method='nearest', fill_value=0.0)
    interp_base = griddata(coords_cart_base, temp_base.flatten(), nodes_cart, method='nearest', fill_value=0.0)

    # Calculate average temperatures at top and base
    avg_top = np.sum(interp_top * weights_norm) * s.max()**2 / (np.pi * s.max()**2)
    avg_base = np.sum(interp_base * weights_norm) * s.max()**2 / (np.pi * s.max()**2)

    # Return the absolute temperature difference
    return abs(avg_top - avg_base)


In [26]:
def find_Nu_TC(uzT_cyl, Delta_T_cyl):
    """
    Compute the Nusselt number (Nu_TC) 

    Args:
        uzT_cyl (ndarray): Convective heat flux 
        Delta_T_cyl (ndarray): Temperature difference

    Returns:
        ndarray: Nusselt number
    """
    # Compute Nusselt number
    return 1 + uzT_cyl / Delta_T_cyl


In [27]:
def process_timesteps():
    """
    Process timesteps to compute properties for spherical shells and cylinders, then save results.

    This function:
    1. Organizes file paths.
    2. Creates spherical and cylindrical grids.
    3. Computes kinetic energy, vorticity energy, length scales, Reynolds number, and Nusselt number.
    4. Interpolates data onto cylindrical grids.
    5. Calculates gradients (optional) and convective heat flux.
    6. Saves results to .npy files.
    """
    start_time = time.process_time()
    
    # settings
    input_path, output_path = basic_settings()
    input_list = create_input_list(input_path)
    norm_path = os.path.normpath(output_path)
    timesteps = []
    results = "output_data"
    filepath_new1 = os.path.join(output_path, results)
    print("Input folder path:", input_path)
    print("Output folder path: ", filepath_new1)
    
    # Create file for all outputs
    date = datetime.now()
    folder =  "Folder_Ra30" + date.strftime('_%Y_%m_%d') # put name in here
    filepath_new = os.path.join(filepath_new1, folder)
    
    
    try:
        os.makedirs(filepath_new, exist_ok = False)
    except OSError as error: 
        folder2 = "Folder_Ra30" + date.strftime('_%Y_%m_%d_') + "duplicate"
        filepath_new = os.path.join(filepath_new1, folder2)
        os.makedirs(filepath_new, exist_ok = True) 
    
    
    

    # Configuration
    order = 10 # Gaussian order
    num =  100 # number of points in each component of cylindrical array
    
    # Initialize result lists
    KE_array, EOV_array, lengthscales_array, Re_array, Nu_array = ([] for _ in range(5))
    KE_tc_array, EOV_tc_array, lengthscales_tc_array, Re_tc_array, Nu_tc_array = ([] for _ in range(5))

    for i, filename in enumerate(input_list):
        #timestep_path = os.path.join(input_path, filename)
        
        timesteps.append(os.path.join(input_path, input_list[i]))

        # Load data
        #coreData = nc.Dataset(timestep_path)
        # extract data from netCDF file
        coreData = nc.Dataset(timesteps[i])
        
        el, r, az, temp, u_r, u_el, u_az, d_T_del, d_T_dr = get_data(coreData)

        # Spherical grid and calculations
        r1, az1, el1, cmb_mask, icb_mask = create_grid(r, az, el)
        KE, EOV, lengthscale, Re = KE_EOV_Re(input_list, r1, az1, el1, u_r, u_el, u_az, r, az, el)
        KE_array.append(KE)
        EOV_array.append(EOV)
        lengthscales_array.append(lengthscale)
        Re_array.append(Re)
        
        Delta_T = DeltaT(temp, r, az, el, r1, az1, el1, cmb_mask, icb_mask, order)
        Nu_shell = find_Nu_global(r, Delta_T)
        Nu_array.append(Nu_shell)

        # Cylindrical grid and interpolation
        s, phi, z = output_grid(r, num)
        (s_tc, phi_tc, z_tc, u_s, u_phi, u_z, r_tc, az_tc, el_tc, ur_tc, uaz_tc, uel_tc, temp_tc,
         d_T_dr_tc, cyl_top_mask, cyl_base_mask) = interpolation_TC_grid(num, r, az, el, temp, u_r, u_el, u_az,\
                                                                         s, phi, z, r1, az1, el1, d_T_dr)
        
        KE_tc, EOV_tc, lengthscale_tc, Re_tc = KE_EOV_Re_cyl(s_tc, phi_tc, el_tc, u_s, u_phi, u_z, s, phi, z)
        KE_tc_array.append(KE_tc)
        EOV_tc_array.append(EOV_tc)
        lengthscales_tc_array.append(lengthscale_tc)
        Re_tc_array.append(Re_tc)
        
        uzT_cyl = find_uzT_cylinder(u_z, temp_tc, s, phi, z, s_tc, phi_tc, z_tc, cyl_top_mask, order)
        Delta_T_cyl = find_DeltaT_cyl(temp_tc, u_z, s, phi, z, s_tc, phi_tc, z_tc, cyl_top_mask, cyl_base_mask, order)
        Nu_TC = find_Nu_TC(uzT_cyl, Delta_T_cyl)
        Nu_tc_array.append(Nu_TC)
        
        
        print(f"File {i + 1} processed")
        
        # Cleanup to free memory
        del coreData, el, r, az, temp, u_r, u_el, u_az, d_T_del, d_T_dr
        del r1, az1, el1, cmb_mask, icb_mask
        del KE, EOV, lengthscale, Re, Delta_T, Nu_shell
        del s, phi, z
        del s_tc, phi_tc, z_tc, u_s, u_phi, u_z, r_tc, az_tc, el_tc, ur_tc, uaz_tc, uel_tc, temp_tc, d_T_dr_tc
        del cyl_top_mask, cyl_base_mask
        del KE_tc, EOV_tc, lengthscale_tc, Re_tc
        del uzT_cyl, Delta_T_cyl, Nu_TC

        gc.collect()

    # Save results
    np.save(os.path.join(filepath_new, 'KE_shell.npy'), KE_array)
    np.save(os.path.join(filepath_new, 'EOV_shell.npy'), EOV_array)
    np.save(os.path.join(filepath_new, 'L_shell.npy'), lengthscales_array)
    np.save(os.path.join(filepath_new, 'Re_shell.npy'), Re_array)
    np.save(os.path.join(filepath_new, 'Nu_shell.npy'), Nu_array)

    np.save(os.path.join(filepath_new, 'KE_cyl.npy'), KE_tc_array)
    np.save(os.path.join(filepath_new, 'EOV_cyl.npy'), EOV_tc_array)
    np.save(os.path.join(filepath_new, 'L_cyl.npy'), lengthscales_tc_array)
    np.save(os.path.join(filepath_new, 'Re_cyl.npy'), Re_tc_array)
    np.save(os.path.join(filepath_new, 'Nu_TC.npy'), Nu_tc_array)
    
    # Print CPU time
    cpu_time = time.process_time() - start_time
    print(f"CPU time taken: {cpu_time} seconds")

In [28]:
process_timesteps()

Input folder path: C:/Users/scjkk/Desktop/Test_data/new_Ra30_files/
Output folder path:  C:/Users/scjkk/Desktop/Test_data/new_Ra30_files/output_data
File 1 processed
CPU time taken: 15.5 seconds
